# Deep learning classifiers for urban sound data

## Bibliotecas necessárias

In [10]:
# Importação de Bibliotecas Necessárias
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import soundata
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Extração de features e download dos sons

In [2]:
# Inicializar e baixar o conjunto de dados UrbanSound8K
dataset = soundata.initialize('urbansound8k')
#dataset.download()  # Baixar o dataset
#dataset.validate()  # Validar a presença de todos os arquivos esperados

# Função para extrair características (exemplo: MFCC)
def extract_features(audio_data, sample_rate):
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T, axis=0)
    return mfccs_processed

# Lista para armazenar as características e rótulos
features = []
labels = []

# Iterar sobre os exemplos do conjunto de dados
for clip in dataset.clip_ids:
    audio_clip = dataset.clip(clip)
    
    # Carregar os dados de áudio e a taxa de amostragem
    audio_data, sample_rate = audio_clip.audio

    # Extrair características
    data = extract_features(audio_data, sample_rate)

    # Adicionar características e rótulo à lista
    features.append(data)
    tags=audio_clip.tags
    labels.append(tags.labels[np.argmax(tags.confidence)])

# Converter as listas em arrays NumPy
features = np.array(features)
labels = np.array(labels)

print(features)
# Normalização dos dados
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Dividir os dados em treino e teste
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


[[-4.3810263e+02  1.8237880e+02 -8.0175827e+01 ... -4.3171391e+00
  -4.4397454e+00 -4.0319953e+00]
 [-2.0318384e+02  1.9251834e+02 -1.5561098e+02 ...  3.9403834e+00
  -1.0747721e+00 -2.1122086e-01]
 [-2.3871693e+02  1.0732930e+02 -3.6585022e+01 ... -5.3374166e+00
  -1.3342643e+00 -2.5891340e+00]
 ...
 [-4.2917862e+02  1.2991762e+02 -8.3397408e+00 ...  2.9807186e+00
  -1.0773857e+00 -6.8800771e-01]
 [-1.6679102e+02  7.2151688e+01 -3.6855282e+01 ...  3.4777620e+00
  -5.8632250e+00 -3.3734778e-01]
 [-3.7684305e+02  1.1927762e+02 -7.5979164e+01 ... -1.3372263e+00
  -7.2432613e-01  2.6398787e+00]]


## Unir features com labels e criar dataset

In [3]:
df_features = pd.DataFrame(features)

# Adicionar os rótulos ao DataFrame
df_features['label'] = labels

unique_labels = df_features['label'].unique()
print("Etiquetas únicas no conjunto de dados:", unique_labels)

Etiquetas únicas no conjunto de dados: ['children_playing' 'gun_shot' 'air_conditioner' 'jackhammer' 'drilling'
 'engine_idling' 'dog_bark' 'siren' 'street_music' 'car_horn']


### PARA TIRAR. EXEMPLO DE AUDIO E LABEL

In [4]:
from IPython.display import Audio
tags = audio_clip.tags

# Se você quiser todas as etiquetas
all_labels = tags.labels

# Se você quiser a etiqueta com maior confiança
max_confidence_index = np.argmax(tags.confidence)
label_with_highest_confidence = tags.labels[max_confidence_index]

print("Todas as etiquetas:", all_labels)
print("Etiqueta com maior confiança:", label_with_highest_confidence)
Audio(data=audio_data, rate=sample_rate)


Todas as etiquetas: ['children_playing']
Etiqueta com maior confiança: children_playing


## Primeiro Modelo Convulsional Neural Network

### Train test split

In [7]:
# Divisão dos dados em treino e teste
from sklearn.model_selection import train_test_split

# Codificação dos rótulos
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)

# Conversão dos rótulos para formato categórico
y_train_categorical = to_categorical(y_train, num_classes=len(unique_labels))
y_test_categorical = to_categorical(y_test, num_classes=len(unique_labels))

### Definir arquitetura 

In [8]:
# Definição da arquitetura do modelo CNN
from tensorflow.keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(features.shape[1], 1)))  # Usando entrada 1D
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(unique_labels), activation='softmax'))  # Número de classes

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 38, 64)            256       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 19, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 17, 128)           24704     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 8, 128)           0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

### Testar modelo

In [9]:
# Treino do modelo
history = model.fit(X_train, y_train_categorical, epochs=10, batch_size=32, validation_data=(X_test, y_test_categorical))

# Avaliação do modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print("Acurácia no conjunto de teste:", test_accuracy)

Epoch 1/10
219/219 [==============================] - 2s 5ms/step - loss: 1.7393 - accuracy: 0.4001 - val_loss: 1.3135 - val_accuracy: 0.5890
Epoch 2/10
219/219 [==============================] - 1s 4ms/step - loss: 1.1634 - accuracy: 0.6143 - val_loss: 0.9428 - val_accuracy: 0.6932
Epoch 3/10
219/219 [==============================] - 1s 4ms/step - loss: 0.8972 - accuracy: 0.7064 - val_loss: 0.7781 - val_accuracy: 0.7544
Epoch 4/10
219/219 [==============================] - 1s 4ms/step - loss: 0.7382 - accuracy: 0.7569 - val_loss: 0.6352 - val_accuracy: 0.7865
Epoch 5/10
219/219 [==============================] - 1s 4ms/step - loss: 0.6282 - accuracy: 0.7918 - val_loss: 0.5576 - val_accuracy: 0.8163
Epoch 6/10
219/219 [==============================] - 1s 4ms/step - loss: 0.5377 - accuracy: 0.8179 - val_loss: 0.5054 - val_accuracy: 0.8392
Epoch 7/10
219/219 [==============================] - 1s 4ms/step - loss: 0.4702 - accuracy: 0.8494 - val_loss: 0.4695 - val_accuracy: 0.8432
Epoch 